In [39]:
!pip install deeppavlov


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip


In [41]:
!pip install pymorphy2

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 9.3 MB/s eta 0:00:00a 0:00:01
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=d9e513eff02970bbdb9f3ef71f2d7321b056a3432565de1245b323c931eb0a53
  Stored in directory: /home/dan005/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt

[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip


In [42]:
import re

import nltk
nltk.download('wordnet')

import pandas as pd
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from deeppavlov import build_model
import pymorphy2

[nltk_data] Downloading package wordnet to /home/dan005/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
df = pd.read_csv('./toxic_comments.csv', 
    engine='python', 
    on_bad_lines='skip',
)

In [5]:
df.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [6]:
df = df.drop('Unnamed: 0', axis=1)

Чтение датасета + удаление ненужного столбца.

In [43]:
lemmatizer = pymorphy2.MorphAnalyzer()
vectorizer = TfidfVectorizer()

In [44]:
def clear_and_lemmatize_series(series: pd.Series) -> pd.Series:
    return series.apply(_clear_and_lemmatize_text)


def _clear_and_lemmatize_text(text: str) -> str:
    return _clear_text(_lemmatize_text(text))
    

def _clear_text(text: str) -> str:
    return re.sub(r'[^\w\s]', ' ', text)


def _lemmatize_text(text: str) -> str:
    return ' '.join(lemmatizer.parse(word)[0].normal_form for word in text.lower().split())


def vectorize_series(series: pd.Series) -> list[str]:
    return list(vectorizer.fit_transform(series))

In [61]:
df['prepared_text'] = clear_and_lemmatize_series(df['text'])

In [62]:
df['vectorized_text'] = vectorize_series(df['text'])

Выполняем минимальную предобработку текстов + лемматизацию.
Также мы сделали векторизацию текстов, но дальше она не используется, поскольку это уже включено в `deeppavlov.build_model`.

In [63]:
df.head()

,text,toxic,prepared_text,vectorized_text
0,Explanation\nWhy the edits made under my usern...,0,explanation why the edits made under my userna...,"(0, 3446)\t0.16769576946881884\n (0, 4373)\..."
1,D'aww! He matches this background colour I'm s...,0,d aww he match this background colour i m see...,"(0, 142295)\t0.16412463608924402\n (0, 2613..."
2,"Hey man, I'm really not trying to edit war. It...",0,hey man i m really not trying to edit war it...,"(0, 69411)\t0.20448767581830993\n (0, 8935)..."
3,"""\nMore\nI can't make any real suggestions on ...",0,more i can t make any real suggestion on imp...,"(0, 137467)\t0.16225009518550423\n (0, 5929..."
4,"You, sir, are my hero. Any chance you remember...",0,you sir are my hero any chance you remember...,"(0, 146787)\t0.18634179087764524\n (0, 1136..."


In [49]:
model = build_model('insults_kaggle_bert', download=True, install=True)


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip


Ignoring transformers: markers 'python_version < "3.8"' don't match your environment



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip
2025-02-08 04:49:39.391 INFO in 'deeppavlov.download'['download'] at line 138: Skipped http://files.deeppavlov.ai/datasets/insults_data.tar.gz download because of matching hashes
2025-02-08 04:49:42.331 INFO in 'deeppavlov.download'['download'] at line 138: Skipped http://files.deeppavlov.ai/deeppavlov_data/classifiers/insults_kaggle_torch_bert_v5.tar.gz download because of matching hashes
/home/dan005/rock_team/venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cl

In [ ]:
df['result'] = model(df['prepared_text'].tolist())

In [ ]:
def unify_result_series(series):
    series = series.str.replace('Not Insult', '0').str.replace('Insult', '1').astype(int)

In [ ]:
df['result'] = unify_result_series(df['result'])

In [ ]:
(df['result'] == df['toxic']).sum() / len(df)

In [46]:
df = pd.read_csv('./rusentitweet_full.csv', 
    engine='python', 
    on_bad_lines='skip',
)

In [47]:
df.head()

,Unnamed: 0,text,label,id
0,0,@varlamov @McFaul На,skip,1327934765807308801
1,1,велл они всё равно что мусор так что ничего с...,negative,1252943181387350017
2,2,"""трезвая жизнь какая-то такая стрёмная""\r\n(с)...",negative,1323610669061677056
3,3,Ой какие неожиданные результаты 🤭 https://t.co...,neutral,1336231661160247297
4,4,@Shvonder_chief @dimsmirnov175 На заборе тоже ...,neutral,1292421736454127617


In [48]:
df = df.drop('Unnamed: 0', axis=1)

In [49]:
df = df.drop('id', axis=1)

In [54]:
df['label'] = df['label'].str.replace('skip', '0').str.replace('neutral', '0').str.replace('speech', '0').str.replace('positive', '0').str.replace('negative', '1').astype(int)

In [51]:
df['prepared_text'] = clear_and_lemmatize_series(df['text'])

In [52]:
df['vectorized_text'] = vectorize_series(df['text'])

In [55]:
df.head()

,text,label,prepared_text,vectorized_text
0,@varlamov @McFaul На,0,varlamov mcfaul на,"(0, 23468)\t0.2461112602099971\n (0, 6947)\..."
1,велл они всё равно что мусор так что ничего с...,1,велла они всё равно что мусор так что ничего с...,"(0, 33023)\t0.3948292135261448\n (0, 24491)..."
2,"""трезвая жизнь какая-то такая стрёмная""\r\n(с)...",1,трезвый жизнь какой то такой стрёмная с ар...,"(0, 2020)\t0.43475956880519917\n (0, 12903)..."
3,Ой какие неожиданные результаты 🤭 https://t.co...,0,ой какой неожиданный результат https t co ...,"(0, 12249)\t0.5238290833865648\n (0, 2828)\..."
4,@Shvonder_chief @dimsmirnov175 На заборе тоже ...,0,shvonder_chief dimsmirnov175 на забор тоже н...,"(0, 15296)\t0.2777613868475538\n (0, 31878)..."


In [12]:
model = build_model('rusentiment_convers_bert', download=True, install=True)


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip


Ignoring transformers: markers 'python_version < "3.8"' don't match your environment



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip
2025-02-08 05:14:20.998 INFO in 'deeppavlov.core.data.utils'['utils'] at line 97: Downloading from http://files.deeppavlov.ai/v1/classifiers/rusentiment_convers_bert/rusentiment_convers_bert_torch.tar.gz to /home/dan005/.deeppavlov/models/classifiers/rusentiment_convers_bert_torch.tar.gz
100%|██████████████████████████████████████████████████████████████████████████████| 1.52G/1.52G [02:16<00:00, 11.1MB/s]
2025-02-08 05:16:38.112 INFO in 'deeppavlov.core.data.utils'['utils'] at line 284: Extracting /home/dan005/.deeppavlov/models/classifiers/rusentiment_convers_bert_torch.tar.gz archive into /home/dan005/.deeppavlov/models/classifiers/rusentiment_convers_bert_torch
/home/dan005/rock_team/venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possib

tokenizer_config.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.40M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased-conversational were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassi

In [ ]:
df['result'] = model(df['prepared_text'].tolist())

In [ ]:
df['result'] = unify_result_series(df['result'])

In [ ]:
(df['result'] == df['label']).sum() / len(df)